In [2]:
import pandas as pd
import numpy as np
from pymatgen.core import Composition
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
import math
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
config = {
    "mathtext.fontset":'stix',
    "font.family":'serif',
    "font.serif": ['Arial'],
    "font.size": 24,
    'axes.unicode_minus': False 
}
plt.rcParams.update(config)
plt.rcParams['axes.unicode_minus'] = False  
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (8, 6),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.rcParams['figure.dpi'] = 400
plt.rcParams['axes.grid'] = False

In [3]:
def get_combined_features(data: pd.DataFrame):
    columns = data.columns
    A_features = []
    B_features = []
    O_features = []
    for column in columns:
        if str(column).__contains__('A'):
            A_features.append(column)
        elif str(column).__contains__('B'):
            B_features.append(column)
        elif str(column).__contains__('O'):
            O_features.append(column)

    for a_feature, b_feature, o_feature in zip(A_features, B_features, O_features):
        feature_name = str(a_feature).split('_')[1]
        feature_name = feature_name
        data['A-B_'+feature_name] = np.abs(data[a_feature] - data[b_feature])
        data['(A+B)-O_'+feature_name] = np.abs((data[a_feature] + data[b_feature])- data[o_feature])


    data = data.drop(columns=O_features)
    return data

In [ ]:
ABO3_ec_magpie_data = pd.read_csv('/data/ABO3_ec_magpie_data.csv').drop(columns = 'Unnamed: 0')
ec_magpie_data = pd.read_csv('/data/ec_magpie_data.csv').drop(columns = 'Unnamed: 0')

In [ ]:
small_data = pd.read_csv('/data/single_perovskite_oxide.csv')
small_data = small_data.drop(columns = 'Unnamed: 0').reset_index(drop=True)
small_data = small_data.drop(columns = 'composition').reset_index(drop=True)
small_data['composition'] = small_data['formula'].map(Composition)
y_eg_abo3 = small_data['target']

In [ ]:
pysic_features_ABO3 = pd.read_csv('/data/pysic_features_ABO3.csv').drop(columns = 'Unnamed: 0')
pysic_labels = pysic_features_ABO3.columns.drop('formula')
pysic_features_ABO3 = pysic_features_ABO3[pysic_labels]
pysic_features_ABO3 = get_combined_features(pysic_features_ABO3)

In [ ]:
re_data = pd.read_csv('/data/all_features_df.csv')
re_data

,formula,A1_x,A1_ratio_x,nA1_x,A2_x,A2_ratio_x,nA2_x,B1_x,B1_ratio_x,B2_x,...,B_atomic_ie_1,B_atomic_ie_2,B_atomic_ndunf,B_atomic_ndval,B_atomic_npunf,B_atomic_npval,B_atomic_nunf,B_atomic_nval,B_atomic_phi,B_atomic_pol
0,Sm0.2Sc0.8Cr1O3,Sm,0.2,3,Sc,0.8,3,Cr,1.0,Cr,...,6.766510,16.485700,5.0,5.0,0.0,0.0,6.0,6.0,4.650,11.600
1,Sm0.2Sc0.8Fe1O3,Sm,0.2,3,Sc,0.8,3,Fe,1.0,Fe,...,7.902400,16.187700,4.0,6.0,0.0,0.0,4.0,8.0,4.930,8.400
2,Sm0.2Zr1Sc0.8O3,Sm,0.2,3,Sc,0.8,3,Zr,1.0,Zr,...,6.633900,13.100000,8.0,2.0,0.0,0.0,8.0,4.0,3.450,17.900
3,Sm0.2Sc0.8Ge1O3,Sm,0.2,3,Sc,0.8,3,Ge,1.0,Ge,...,7.899430,15.934610,0.0,10.0,4.0,2.0,4.0,14.0,4.550,5.840
4,Sm0.2Hf1Sc0.8O3,Sm,0.2,3,Sc,0.8,3,Hf,1.0,Hf,...,6.825070,15.000000,8.0,2.0,0.0,0.0,8.0,18.0,3.600,16.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,Sr1Ce0.7Zr0.3O3,Sr,1.0,2,Sr,0.0,2,Ce,0.7,Zr,...,5.867260,11.525000,8.7,1.3,0.0,0.0,17.8,4.0,3.261,26.090
35321,Sr1Ce0.7Be0.3O3,Sr,1.0,2,Sr,0.0,2,Ce,0.7,Be,...,6.673900,13.058342,6.3,0.7,0.0,0.0,15.4,3.4,3.741,22.400
35322,Sr1Ce0.7Ge0.3O3,Sr,1.0,2,Sr,0.0,2,Ce,0.7,Ge,...,6.246919,12.375383,6.3,3.7,1.2,0.6,16.6,7.0,3.591,22.472
35323,Sr1Ce0.7Hf0.3O3,Sr,1.0,2,Sr,0.0,2,Ce,0.7,Hf,...,5.924611,12.095000,8.7,1.3,0.0,0.0,17.8,8.2,3.306,25.580


In [9]:
def get_combined_features(data: pd.DataFrame):
    columns = data.columns
    A_features = []
    B_features = []
    O_features = []
    for column in columns:
        if str(column).__contains__('A'):
            A_features.append(column)
        elif str(column).__contains__('B'):
            B_features.append(column)
        elif str(column).__contains__('O'):
            O_features.append(column)

    for a_feature, b_feature, o_feature in zip(A_features, B_features, O_features):
        feature_name = str(a_feature).split('_')[1]
        feature_name = feature_name
        data['A-B_'+feature_name] = np.abs(data[a_feature] - data[b_feature])
        data['(A+B)-O_'+feature_name] = np.abs((data[a_feature] + data[b_feature])- data[o_feature])


    data = data.drop(columns=O_features)
    return data

In [10]:
pysic_features = re_data[pysic_labels]
pysic_features =  get_combined_features(pysic_features)
pysic_features[['formula','Eg']] = re_data[['formula','Eg']]
pysic_features

,A_Density,B_Density,A_dipole Polarizability,B_dipole Polarizability,A_covalent Radius,B_covalent Radius,A_atomic Radius,B_atomic Radius,A_FirstIonization,B_FirstIonization,...,A-B_Electronegativity,(A+B)-O_Electronegativity,A-B_number of s+p Electrons,(A+B)-O_number of s+p Electrons,A-B_number of d Electrons,(A+B)-O_number of d Electrons,A-B_Mulliken EN,(A+B)-O_Mulliken EN,formula,Eg
0,3.896,7.150,116.0,83.000,152.8,122.0,1.65,1.400,615.38,652.90,...,0.338,0.458,1.0,3.0,4.2,5.8,0.428,0.528,Sm0.2Sc0.8Cr1O3,2.146025
1,3.896,7.870,116.0,62.000,152.8,116.0,1.65,1.400,615.38,762.50,...,0.508,0.288,0.0,2.0,5.2,6.8,0.768,0.188,Sm0.2Sc0.8Fe1O3,1.374682
2,3.896,6.510,116.0,112.000,152.8,154.0,1.65,1.550,615.38,640.10,...,0.008,0.788,0.0,2.0,1.2,2.8,0.348,0.608,Sm0.2Zr1Sc0.8O3,4.177025
3,3.896,5.320,116.0,40.000,152.8,121.0,1.65,1.250,615.38,762.00,...,0.688,0.108,2.0,0.0,0.8,0.8,1.308,0.352,Sm0.2Sc0.8Ge1O3,3.941249
4,3.896,13.300,116.0,103.000,152.8,152.0,1.65,1.550,615.38,658.50,...,0.022,0.818,0.0,2.0,1.2,2.8,0.508,0.448,Sm0.2Hf1Sc0.8O3,4.529786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,2.640,6.692,197.2,177.100,185.0,160.3,2.00,1.760,549.50,566.11,...,0.233,1.307,0.0,2.0,1.3,1.3,1.262,2.278,Sr1Ce0.7Zr0.3O3,3.843865
35321,2.640,5.294,197.2,154.822,185.0,144.7,2.00,1.610,549.50,643.93,...,0.305,1.235,0.0,2.0,0.7,0.7,1.640,1.900,Sr1Ce0.7Be0.3O3,3.431016
35322,2.640,6.335,197.2,155.500,185.0,150.4,2.00,1.670,549.50,602.68,...,0.437,1.103,0.6,1.4,0.7,0.7,1.550,1.990,Sr1Ce0.7Ge0.3O3,2.085930
35323,2.640,8.729,197.2,174.400,185.0,159.7,2.00,1.760,549.50,571.63,...,0.224,1.316,0.0,2.0,1.3,1.3,1.310,2.230,Sr1Ce0.7Hf0.3O3,3.646532


In [11]:
p_label = pysic_features.columns.drop(['formula', 'Eg'])
p_label

Index(['A_Density', 'B_Density', 'A_dipole Polarizability',
       'B_dipole Polarizability', 'A_covalent Radius', 'B_covalent Radius',
       'A_atomic Radius', 'B_atomic Radius', 'A_FirstIonization',
       'B_FirstIonization', 'A_number of Valence Electrons',
       'B_number of Valence Electrons', 'A_number', 'B_number', 'A_Period',
       'B_Period', 'A_Electronegativity', 'B_Electronegativity',
       'A_number of s+p Electrons', 'B_number of s+p Electrons',
       'A_number of d Electrons', 'B_number of d Electrons', 'A_Mulliken EN',
       'B_Mulliken EN', 'A-B_Density', '(A+B)-O_Density',
       'A-B_dipole Polarizability', '(A+B)-O_dipole Polarizability',
       'A-B_covalent Radius', '(A+B)-O_covalent Radius', 'A-B_atomic Radius',
       '(A+B)-O_atomic Radius', 'A-B_FirstIonization',
       '(A+B)-O_FirstIonization', 'A-B_number of Valence Electrons',
       '(A+B)-O_number of Valence Electrons', 'A-B_number', '(A+B)-O_number',
       'A-B_Period', '(A+B)-O_Period', 'A-B_El

In [12]:
from scipy.stats import spearmanr
def remove_collinear_features(x, y,threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model
        to generalize and improves the interpretability of the model.
        
    Inputs: 
        threshold: any features with correlations greater than this value are removed
    
    Output: 
        dataframe that contains only the non-highly-collinear features
    '''  
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
#                 print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                x1 = x[col.values[0]]
                x2 = x[row.values[0]]
                corr1,p1 = spearmanr(x1, y)
                corr2,p2 = spearmanr(x2, y)
                if np.abs(corr1) < np.abs(corr2):
                    drop_cols.append(col.values[0])
                else:
                    drop_cols.append(row.values[0])
    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    return x

# EC + mgapie + physic

In [13]:
data = ec_magpie_data.copy(deep = True)
data[p_label] = pysic_features[p_label]
data

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_val_neg_1,...,A-B_Period,(A+B)-O_Period,A-B_Electronegativity,(A+B)-O_Electronegativity,A-B_number of s+p Electrons,(A+B)-O_number of s+p Electrons,A-B_number of d Electrons,(A+B)-O_number of d Electrons,A-B_Mulliken EN,(A+B)-O_Mulliken EN
0,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,0.4,6.4,0.338,0.458,1.0,3.0,4.2,5.8,0.428,0.528
1,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,0.4,6.4,0.508,0.288,0.0,2.0,5.2,6.8,0.768,0.188
2,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,0.6,7.4,0.008,0.788,0.0,2.0,1.2,2.8,0.348,0.608
3,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,0.4,6.4,0.688,0.108,2.0,0.0,0.8,0.8,1.308,0.352
4,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,1.6,8.4,0.022,0.818,0.0,2.0,1.2,2.8,0.508,0.448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,0.7,8.7,0.233,1.307,0.0,2.0,1.3,1.3,1.262,2.278
35321,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,0.2,7.8,0.305,1.235,0.0,2.0,0.7,0.7,1.640,1.900
35322,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,0.4,8.4,0.437,1.103,0.6,1.4,0.7,0.7,1.550,1.990
35323,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,1.0,9.0,0.224,1.316,0.0,2.0,1.3,1.3,1.310,2.230


In [28]:
# data.to_csv('all_features.csv')

In [14]:
X = data.copy(deep = True)
y = re_data['Eg']
X = remove_collinear_features(X,y,0.8)
fl = X.columns
X = data[fl]
X

,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_r_val,A_atomic_r_s_1,A_atomic_r_d_1,A_atomic_r_val_1,A_atomic_ie_2,...,A-B_FirstIonization,A-B_number of Valence Electrons,(A+B)-O_number of Valence Electrons,A-B_Period,A-B_Electronegativity,A-B_number of s+p Electrons,(A+B)-O_number of s+p Electrons,A-B_number of d Electrons,(A+B)-O_number of d Electrons,(A+B)-O_Mulliken EN
0,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,37.52,3.0,3.0,0.4,0.338,1.0,3.0,4.2,5.8,0.528
1,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,147.12,5.0,5.0,0.4,0.508,0.0,2.0,5.2,6.8,0.188
2,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,24.72,1.0,1.0,0.6,0.008,0.0,2.0,1.2,2.8,0.608
3,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,146.62,1.0,1.0,0.4,0.688,2.0,0.0,0.8,0.8,0.352
4,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,43.12,1.0,1.0,1.6,0.022,0.0,2.0,1.2,2.8,0.448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,16.61,1.3,0.7,0.7,0.233,0.0,2.0,1.3,1.3,2.278
35321,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,94.43,0.7,1.3,0.2,0.305,0.0,2.0,0.7,0.7,1.900
35322,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,53.18,1.3,0.7,0.4,0.437,0.6,1.4,0.7,0.7,1.990
35323,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,22.13,1.3,0.7,1.0,0.224,0.0,2.0,1.3,1.3,2.230


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [16]:
X_test

,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_r_val,A_atomic_r_s_1,A_atomic_r_d_1,A_atomic_r_val_1,A_atomic_ie_2,...,A-B_FirstIonization,A-B_number of Valence Electrons,(A+B)-O_number of Valence Electrons,A-B_Period,A-B_Electronegativity,A-B_number of s+p Electrons,(A+B)-O_number of s+p Electrons,A-B_number of d Electrons,(A+B)-O_number of d Electrons,(A+B)-O_Mulliken EN
33436,-1.201110,-1.201110,-1.201110,1.590598,5.774711,0.33765,1.84795,0.00000,0.33695,10.900000,...,129.27,0.7,0.7,1.6,0.725,1.7,0.3,7.0,7.0,0.461
15481,-3.553540,-0.005668,-3.553540,0.866903,5.865687,0.35607,0.00000,0.00000,0.30230,11.950900,...,13.34,0.0,0.0,3.0,0.375,1.0,1.0,0.1,0.1,1.210
428,-1.967122,-1.657150,-1.657150,-0.073590,3.793139,2.12911,1.01933,0.52499,1.11853,20.943952,...,233.66,4.6,1.4,2.0,0.788,0.2,3.8,4.8,5.2,1.456
7241,-2.375221,-1.768219,-1.768219,0.236986,4.210911,2.47131,1.19918,0.00000,1.28598,19.211357,...,244.64,2.7,0.7,0.0,0.480,0.7,2.7,2.0,2.0,1.494
3707,-3.707976,-3.707976,-3.707976,-0.788380,6.879914,1.36765,1.09247,0.48441,1.14339,16.001252,...,134.28,2.4,2.4,1.0,0.144,0.4,0.4,10.0,10.0,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,-2.179162,-2.179162,-2.179162,0.730043,5.385046,0.54519,0.00000,0.00000,1.12699,12.349816,...,96.72,1.0,1.0,0.6,0.646,2.0,0.0,9.2,10.8,0.052
9237,-2.230671,-2.230671,-2.230671,0.768554,5.425322,1.25958,0.00000,0.51626,1.06374,17.049141,...,1.94,0.0,0.0,0.7,0.156,0.3,0.3,7.3,7.3,1.168
29851,-1.201110,-1.201110,-1.201110,1.590598,5.774711,0.33765,1.84795,0.00000,0.33695,10.900000,...,263.10,0.2,0.2,2.4,0.332,0.0,2.0,0.8,0.8,0.020
8904,-2.116304,-2.116304,-2.116304,-1.042984,3.552501,1.67504,0.96559,0.00000,0.99457,25.306000,...,207.54,2.4,0.4,0.6,0.627,0.7,2.7,2.0,2.0,1.466


In [17]:
# index_test = X_test.index
# test_df = data.iloc[index_test]
# test_df['Eg'] = y_test
# test_df.to_csv('test_df.csv')

In [18]:
# index_train = X_train.index
# train_df = data.iloc[index_train]
# train_df['Eg'] = y_train
# train_df.to_csv('train_df.csv')

In [19]:
dop_magpie_etr = ExtraTreesRegressor(n_jobs = -1 ,random_state=42)
dop_magpie_etr.fit(X_train, y_train)
y_pred = dop_magpie_etr.predict(X_test)
print('Performance on the test set: MAE = %0.4f' % mean_absolute_error(y_test,y_pred))
print('Performance on the test set: RMSE = %0.4f' % np.sqrt(mean_squared_error(y_test, y_pred)))
print('Performance on the test set: R2 SCORE = %0.4f' % r2_score(y_test,y_pred))

Performance on the test set: MAE = 0.3089
Performance on the test set: RMSE = 0.4242
Performance on the test set: R2 SCORE = 0.8826


In [18]:
from sklearn.ensemble import ExtraTreesRegressor
etr = ExtraTreesRegressor(n_jobs=-1, random_state=42)
etr.fit(X_train, y_train)
feature_importance=pd.DataFrame()
feature_importance['features'] = X.columns
feature_importance['feature_importances'] = etr.feature_importances_
threshold = np.sort(feature_importance['feature_importances'].values)
score = []
n_features = []
cols = []
for i in tqdm(threshold):
    selector = SelectFromModel(etr,threshold=i)
    X_embedded = selector.fit_transform(X_train, y_train)
    once = cross_val_score(etr,X_embedded,y_train,cv=5,scoring="r2",n_jobs=-1).mean()#交叉验证
    score.append(once)
    n_features.append(X_embedded.shape[1])
    cols.append(X.columns[selector.get_support()])
result = pd.DataFrame({"col":cols,"score":score,"n_features":n_features})
result

100%|██████████| 121/121 [08:49<00:00,  4.38s/it]


,col,score,n_features
0,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.870533,121
1,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.870533,121
2,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.870533,121
3,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.870533,121
4,"Index(['A_atomic_hpomo', 'A_atomic_lfumo', 'A_...",0.870533,121
...,...,...,...
116,"Index(['MagpieData maximum NUnfilled', 'B_numb...",0.703090,5
117,"Index(['MagpieData maximum NUnfilled', 'B_numb...",0.689762,4
118,"Index(['MagpieData maximum NUnfilled', 'B_numb...",0.581654,3
119,"Index(['MagpieData maximum NUnfilled', 'B_numb...",0.490258,2


In [19]:
features_select = result.sort_values(by='score',ascending=True)
features_select = features_select[features_select['score']>=0.87]

In [20]:
optimize_feature_labels = features_select.iloc[0]['col']
X_train = X.loc[X_train.index][optimize_feature_labels]
X_test = X.loc[X_test.index][optimize_feature_labels]
if 'O' in X_train.columns:
    X_train = X_train.drop(columns='O')
    X_test = X_test.drop(columns='O')
X_train.shape, X_test.shape

((28260, 62), (7065, 62))

In [21]:
ABO3_data = pysic_features_ABO3
ABO3_ec_magpie_p_data = ABO3_ec_magpie_data.copy(deep = True)
ABO3_ec_magpie_p_data[p_label] = ABO3_data[p_label]
ABO3_ec_magpie_p_data

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_val_neg_1,...,A-B_Period,(A+B)-O_Period,A-B_Electronegativity,(A+B)-O_Electronegativity,A-B_number of s+p Electrons,(A+B)-O_number of s+p Electrons,A-B_number of d Electrons,(A+B)-O_number of d Electrons,A-B_Mulliken EN,(A+B)-O_Mulliken EN
0,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,1.50475,...,2,8,0.49,0.61,1,3,5,5,0.62,0.72
1,-4.38669,-2.25963,-2.25963,-2.25963,1.578750,6.198956,1.68613,7.08536,1.84895,0.64805,...,0,6,0.52,0.20,0,2,6,8,0.96,0.10
2,-2.01997,-2.01997,-0.47011,-0.47011,0.846499,4.172046,2.68397,6.53600,1.94775,1.37595,...,2,8,0.46,0.58,1,3,4,6,0.62,0.72
3,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,1,7,0.44,0.56,1,3,4,6,0.53,0.63
4,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,2,8,0.73,0.51,0,2,5,7,0.96,0.38
5,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,1.50475,...,2,8,0.71,0.39,0,2,7,7,1.20,0.14
6,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,1,7,0.61,0.39,0,2,5,7,0.87,0.29
7,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,2,8,0.56,0.68,1,3,4,6,0.62,0.72
8,-3.55283,-1.85729,-1.85729,-1.85729,-2.664785,2.129405,0.60005,6.41133,2.09365,0.34455,...,2,8,0.53,0.65,1,3,5,5,0.62,0.72
9,-2.78148,-2.78148,-1.07232,-1.07232,2.903783,6.083156,1.87232,6.37620,2.11275,1.31275,...,2,8,0.52,0.64,1,3,5,5,0.62,0.72


In [39]:
# ABO3_ec_magpie_p_data.to_csv('ABO3_all_features.csv')

In [23]:
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tqdm import tqdm
train_model = ExtraTreesRegressor(n_jobs = -1, random_state = 42)
y_eg_abo3 = small_data['target']
error1 = []
error2 = []
r2_prm = []
error1_base=[]
error2_base=[]
r2_prl=[]
train_set = []
test_set = []
tmp_df = pd.DataFrame()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_indices = kf.split(X,y)
base_features = ABO3_ec_magpie_p_data[optimize_feature_labels]
for i,(train_index, test_index) in enumerate(cv_indices):
    train_set.append(train_index)
    test_set.append(test_index)
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    X_train = X_train[optimize_feature_labels]
    X_test = X_test[optimize_feature_labels]
    name = f"fold_{i}"
    train_model.fit(X_train, y_train)
    y_pred =  train_model.predict(X_test)
    mae_model = mean_absolute_error(y_test, y_pred)
    rmse_model = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_model = r2_score(y_test,y_pred)
    error1.append(mae_model)
    error2.append(rmse_model)
    r2_prm.append(r2_model)
    
    y_base = train_model.predict(base_features)
    tmp_df[f'column_{i}'] = y_base
    MAE_BASE = mean_absolute_error(y_base,y_eg_abo3)
    RMSE_BASE = np.sqrt(mean_squared_error(y_base,y_eg_abo3))
    R2_BASE = r2_score(y_base,y_eg_abo3)
    error1_base.append(MAE_BASE)
    error2_base.append(RMSE_BASE)
    r2_prl.append(R2_BASE)
    print(name)
    print(f"Performance on the test set: MAE = %0.4f,RMSE = %0.4f, R2 = %0.4f" % (mae_model, rmse_model, r2_model))
    print(f"Performance on the small set: MAE = %0.4f,RMSE = %0.4f, R2 = %0.4f" % (MAE_BASE, RMSE_BASE, R2_BASE))
mean_error1 = sum(error1) / len(error1)
mean_error2 = sum(error2) / len(error2)
mean_r2 = sum(r2_prm) / len(r2_prm)
print('Performance on the test set using 5-fold: MAE = %0.4f' % mean_error1)
print('Performance on the test set using 5-fold: RMSE = %0.4f' % mean_error2)
print('Performance on the test set using 5-fold: R2 = %0.4f' % mean_r2)

mean_error1_base = sum(error1_base) / len(error1_base)
mean_error2_base = sum(error2_base) / len(error2_base)
mean_r2_base = sum(r2_prl) / len(r2_prl)
print('Performance on the valid set using 5-fold: MAE = %0.4f' % mean_error1_base)
print('Performance on the valid set using 5-fold: RMSE = %0.4f' % mean_error2_base)
print('Performance on the valid set using 5-fold: R2 = %0.4f' % mean_r2_base)

fold_0
Performance on the test set: MAE = 0.3108,RMSE = 0.4277, R2 = 0.8807
Performance on the small set: MAE = 0.4465,RMSE = 0.5575, R2 = 0.6142
fold_1
Performance on the test set: MAE = 0.3177,RMSE = 0.4369, R2 = 0.8746
Performance on the small set: MAE = 0.4344,RMSE = 0.5485, R2 = 0.6322
fold_2
Performance on the test set: MAE = 0.3172,RMSE = 0.4358, R2 = 0.8794
Performance on the small set: MAE = 0.4390,RMSE = 0.5627, R2 = 0.6276
fold_3
Performance on the test set: MAE = 0.3121,RMSE = 0.4285, R2 = 0.8787
Performance on the small set: MAE = 0.4386,RMSE = 0.5565, R2 = 0.6473
fold_4
Performance on the test set: MAE = 0.3088,RMSE = 0.4237, R2 = 0.8820
Performance on the small set: MAE = 0.4375,RMSE = 0.5517, R2 = 0.6495
Performance on the test set using 5-fold: MAE = 0.3133
Performance on the test set using 5-fold: RMSE = 0.4305
Performance on the test set using 5-fold: R2 = 0.8791
Performance on the valid set using 5-fold: MAE = 0.4392
Performance on the valid set using 5-fold: RMSE =

In [24]:
best_train_index = train_set[4]
best_test_index = test_set[4]
best_X_train = X.loc[best_train_index].reset_index(drop=True)
best_X_test = X.loc[best_test_index].reset_index(drop=True)
best_y_train = y.loc[best_train_index].reset_index(drop=True)
best_y_test = y.loc[best_test_index].reset_index(drop=True)

In [26]:
best_X_train = best_X_train[optimize_feature_labels]
best_X_test = best_X_test[optimize_feature_labels]

In [35]:
model_best = ExtraTreesRegressor(n_jobs=-1, random_state=42)
model_best.fit(best_X_train, best_y_train)
y_pred = model_best.predict(best_X_test)
MAE_4 = mean_absolute_error(best_y_test,y_pred)
RMSE_4 = np.sqrt(mean_squared_error(best_y_test, y_pred))
R2_4 = r2_score(best_y_test,y_pred)
print('Performance on the test set: MAE = %0.4f' % MAE_4)
print('Performance on the test set: RMSE = %0.4f' % RMSE_4)
print('Performance on the test set: R2 SCORE = %0.4f' % R2_4)

Performance on the test set: MAE = 0.3088
Performance on the test set: RMSE = 0.4237
Performance on the test set: R2 SCORE = 0.8820


In [29]:
best_f = optimize_feature_labels.tolist()

In [30]:
# import json
# with open("best_feature_m_p_ec.json", "w") as f:
#     json.dump(best_f, f)

In [36]:
# import pickle
# with open('model_m_ec_p.pkl', 'wb') as f:
#     pickle.dump(model_best, f)